In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Baseline model

In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=bdf4ce9eacdd6e14978293142620a00be73e38f82d9e0987e5226a6be4f7a990
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
#python librairies
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer
import pickle
from pickle import dump
import re

In [4]:
model_bert = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
multilabel_binarizer = pickle.load(open("/content/drive/MyDrive/Openclassrooms/projet7/multilabel_binarizer.pkl", 'rb'))
loaded_model = pickle.load(open("/content/drive/MyDrive/Openclassrooms/projet7/trained_model-0.1.0.pkl", 'rb'))

In [6]:
def clean_sentence(text):
    stop_words = stopwords.words('english')
    my_punctuation = '!"$%&\()*,-./:;<=>?@[\\]^_`{|}~'
    #text lowercase
    text = text.lower()
    #remove punctiation in except + and # for C++ and C#
    text = text.translate(str.maketrans("", "", my_punctuation))
    #remove numbers
    text = re.sub('\d+', '', text)
    # Remove unicode characters
    text = text.encode("ascii", "ignore").decode()
    # Remove extra spaces
    text = re.sub('\s+', ' ', text)
    #tokenization
    #text = text.split()
    #remove_stop_words
    #text = [word for word in text.split() if not word in stop_words]
    sent = []
    sent.append(text)
    return sent

In [7]:
def get_tags(text):
    # p = input("Enter your sentence: ") # raw_input() function

    start_time= time.time() # set the time at which inference started
    sentence = clean_sentence(text)
    sentence_embeddings = model_bert.encode(sentence)
    y_pred = loaded_model.predict(sentence_embeddings)
    tag = multilabel_binarizer.inverse_transform(y_pred)
    stop_time=time.time()
    duration =stop_time - start_time
    hours = duration // 3600
    minutes = (duration - (hours * 3600)) // 60
    seconds = duration - ((hours * 3600) + (minutes * 60))
    print(f'inference elapsed time was {str(hours)} hours, {minutes:4.1f} minutes,{seconds:4.2f} seconds)', flush=True) # print out inferenceduration time
    return tag

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
import time
text = "how can i make the win api window more modern looking i ordered programming windows fifth edition a few days ago and started working with it i'm starting to learn the win api however i got a question the windows do not look modern winxpwin vistawin style at all how do i fix this it currently looks like this crap font and all thanks in advance machiel"
real_tags = ['c', 'c++', 'windows']
tags = get_tags(text)
print("predicted Tags : ",tags)
print("real tags :",real_tags)

inference elapsed time was 0.0 hours,  0.0 minutes,0.17 seconds)
predicted Tags :  [('windows',)]
real tags : ['c', 'c++', 'windows']


In [12]:
text = "hide virtual function with nonvirtual override having the output is but i expected i didn't declare with the modifier so i expect the function call to be determined by the static type no polymorphism why is being called is it possible to provide a nonvirtual function in a derived class that hides the virtual function in the base what signature should the derived member function have so that calls it and not"
real_tags = ['c++']
tags = get_tags(text)
print("predicted Tags : ",tags)
print("real tags :",real_tags)

inference elapsed time was 0.0 hours,  0.0 minutes,0.11 seconds)
predicted Tags :  [('c++',)]
real tags : ['c++']


In [13]:
text = "url encoded forward slashes breaking my codeigniter app im trying to create a url string that works like this so in other words i then retrieve the url with the forward slashes in the url will of course be a problem accessing uri segments so ill go ahead and url encode the url portion but now i just get a saying it appears that my url encoding of forward slashes breaks cis uri parser what can i do to get around this problem"
real_tags =  ['php']
tags = get_tags(text)
print("predicted Tags : ",tags)
print("real tags :",real_tags)

inference elapsed time was 0.0 hours,  0.0 minutes,0.08 seconds)
predicted Tags :  [('php',)]
real tags : ['php']


# Selected Model

In [2]:
!pip install -q datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00


In [3]:
import os
# Hack for my machine, and Windows
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
from datasets import DatasetDict
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model


# Load Dataset

In [4]:
df = pd.read_feather('/content/drive/MyDrive/Openclassrooms/projet7/cleaned_data_stackoverflow_questions.feather')
df.head()

,Title_cleaned,Body_cleaned,Tags_cleaned,Title_tokenized,Body_tokenized,Tags_tokenized,Tags_list,number_of_tags,Id,Score,ViewCount,AnswerCount
0,when porting java code to objc how best to rep...,i am working on porting a java codebase to coc...,java objective-c cocoa macos porting,"[porting, java, code, objc, best, represent, c...","[working, porting, java, codebase, cocoaobject...","[java, objective-c, cocoa, macos, porting]","[java, objective-c, macos]",3,1117384,8,1346,6
1,is there a way for hiding some enum values for...,i have enum lets say for example and have two ...,c# .net winforms windows-forms-designer prope...,"[way, hiding, enum, values, specific, property...","[enum, lets, say, example, two, classes, prope...","[c#, .net, winforms, windows-forms-designer, p...","[c#, .net]",2,59024032,8,2172,1
2,fixing words with spaces using a dictionary lo...,i have extracted the list of sentences from a ...,python python-2.7 dictionary nltk text-segmen...,"[fixing, words, spaces, using, dictionary, loo...","[extracted, list, sentences, document, preproc...","[python, python-2.7, dictionary, nltk, text-se...",[python],1,19675106,19,6507,7
3,how to split a string at line breaks in python,i want to copy some tabular data from excel in...,python arrays list clipboard pywin32,"[split, string, line, breaks, python]","[want, copy, tabular, data, excel, python, arr...","[python, arrays, list, clipboard, pywin32]","[python, arrays]",2,21205074,6,7848,5
4,how do you express binary literals in python,how do you express an integer as a binary numb...,python syntax binary integer literals,"[express, binary, literals, python]","[express, integer, binary, number, python, lit...","[python, syntax, binary, integer, literals]",[python],1,1476,394,317942,8


In [ ]:
df["Tags_listtext"] = [', '.join(map(str, l)) for l in df["Tags_list"]]

In [ ]:
df['text_sentences'] = df['Title_cleaned']+' '+ df['Body_cleaned']

In [ ]:
df = df[['text_sentences','Tags_listtext','Tags_list']].copy()

In [ ]:
df_labelohe =df['Tags_listtext'].str.get_dummies(sep = ', ')

In [ ]:
df_labelohe

,.net,ajax,algorithm,android,angular,arrays,asp.net,asp.net-mvc,c,c#,...,sql-server,string,swift,typescript,unit-testing,visual-studio,windows,wpf,xcode,xml
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43797,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43798,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43799,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data = df[['text_sentences']].copy()
data.head()

,text_sentences
0,when porting java code to objc how best to rep...
1,is there a way for hiding some enum values for...
2,fixing words with spaces using a dictionary lo...
3,how to split a string at line breaks in python...
4,how do you express binary literals in python h...


In [ ]:
data = pd.concat([data,df_labelohe], axis=1)

In [ ]:
data.head()

,text_sentences,.net,ajax,algorithm,android,angular,arrays,asp.net,asp.net-mvc,c,...,sql-server,string,swift,typescript,unit-testing,visual-studio,windows,wpf,xcode,xml
0,when porting java code to objc how best to rep...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,is there a way for hiding some enum values for...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,fixing words with spaces using a dictionary lo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,how to split a string at line breaks in python...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,how do you express binary literals in python h...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dataset =Dataset.from_pandas(data)

In [ ]:
# 80% train, 20% test + validation
train_testvalid = dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text_sentences', '.net', 'ajax', 'algorithm', 'android', 'angular', 'arrays', 'asp.net', 'asp.net-mvc', 'c', 'c#', 'c++', 'c++11', 'css', 'database', 'django', 'gcc', 'html', 'image', 'ios', 'iphone', 'java', 'javascript', 'jquery', 'json', 'linux', 'macos', 'multithreading', 'mysql', 'node.js', 'objective-c', 'performance', 'php', 'python', 'python-3.x', 'r', 'reactjs', 'ruby', 'ruby-on-rails', 'spring', 'sql', 'sql-server', 'string', 'swift', 'typescript', 'unit-testing', 'visual-studio', 'windows', 'wpf', 'xcode', 'xml'],
        num_rows: 35040
    })
    test: Dataset({
        features: ['text_sentences', '.net', 'ajax', 'algorithm', 'android', 'angular', 'arrays', 'asp.net', 'asp.net-mvc', 'c', 'c#', 'c++', 'c++11', 'css', 'database', 'django', 'gcc', 'html', 'image', 'ios', 'iphone', 'java', 'javascript', 'jquery', 'json', 'linux', 'macos', 'multithreading', 'mysql', 'node.js', 'objective-c', 'performance', 'php', 'python',

In [ ]:
dataset["train"] = dataset["train"].shuffle()
dataset["validation"] = dataset["validation"].shuffle()

In [ ]:
example = dataset['train'][0]
example

{'text_sentences': "ie throwing 'timezone' is outside of valid range when setting timezone to europelondon i have some js to get the local time displayed in uk's timezone works fine in all browsers except ie sigh which throws the following error does anybody have any suggestions see httpjsbincomdawaqocuviedithtmljsconsoleoutput for an example fiddle that supports ie ",
 '.net': 0,
 'ajax': 0,
 'algorithm': 0,
 'android': 0,
 'angular': 0,
 'arrays': 0,
 'asp.net': 0,
 'asp.net-mvc': 0,
 'c': 0,
 'c#': 0,
 'c++': 0,
 'c++11': 0,
 'css': 0,
 'database': 0,
 'django': 0,
 'gcc': 0,
 'html': 0,
 'image': 0,
 'ios': 0,
 'iphone': 0,
 'java': 0,
 'javascript': 1,
 'jquery': 0,
 'json': 0,
 'linux': 0,
 'macos': 0,
 'multithreading': 0,
 'mysql': 0,
 'node.js': 0,
 'objective-c': 0,
 'performance': 0,
 'php': 0,
 'python': 0,
 'python-3.x': 0,
 'r': 0,
 'reactjs': 0,
 'ruby': 0,
 'ruby-on-rails': 0,
 'spring': 0,
 'sql': 0,
 'sql-server': 0,
 'string': 0,
 'swift': 0,
 'typescript': 0,
 'unit

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in [ 'text_sentences']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels


['.net',
 'ajax',
 'algorithm',
 'android',
 'angular',
 'arrays',
 'asp.net',
 'asp.net-mvc',
 'c',
 'c#',
 'c++',
 'c++11',
 'css',
 'database',
 'django',
 'gcc',
 'html',
 'image',
 'ios',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'json',
 'linux',
 'macos',
 'multithreading',
 'mysql',
 'node.js',
 'objective-c',
 'performance',
 'php',
 'python',
 'python-3.x',
 'r',
 'reactjs',
 'ruby',
 'ruby-on-rails',
 'spring',
 'sql',
 'sql-server',
 'string',
 'swift',
 'typescript',
 'unit-testing',
 'visual-studio',
 'windows',
 'wpf',
 'xcode',
 'xml']

In [ ]:
id2label

{0: '.net',
 1: 'ajax',
 2: 'algorithm',
 3: 'android',
 4: 'angular',
 5: 'arrays',
 6: 'asp.net',
 7: 'asp.net-mvc',
 8: 'c',
 9: 'c#',
 10: 'c++',
 11: 'c++11',
 12: 'css',
 13: 'database',
 14: 'django',
 15: 'gcc',
 16: 'html',
 17: 'image',
 18: 'ios',
 19: 'iphone',
 20: 'java',
 21: 'javascript',
 22: 'jquery',
 23: 'json',
 24: 'linux',
 25: 'macos',
 26: 'multithreading',
 27: 'mysql',
 28: 'node.js',
 29: 'objective-c',
 30: 'performance',
 31: 'php',
 32: 'python',
 33: 'python-3.x',
 34: 'r',
 35: 'reactjs',
 36: 'ruby',
 37: 'ruby-on-rails',
 38: 'spring',
 39: 'sql',
 40: 'sql-server',
 41: 'string',
 42: 'swift',
 43: 'typescript',
 44: 'unit-testing',
 45: 'visual-studio',
 46: 'windows',
 47: 'wpf',
 48: 'xcode',
 49: 'xml'}

In [ ]:
from transformers import AutoTokenizer
from transformers import CanineTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("google/canine-s")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text_sentences"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/35040 [00:00<?, ? examples/s]

Map:   0%|          | 0/4381 [00:00<?, ? examples/s]

Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['test'][4]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

"[CLS]raii in ruby or how to manage resources in ruby i know it's by design that you can't control what happens when an object is destroyed i am also aware of defining some class method as a finalizer however is the ruby idiom for c++'s raii resources are initialized in constructor closed in destructor how do people manage resources used inside objects even when errors or exceptions happen using ensure works but users of the class have to remember to do the whole beginrescueensure chacha everytime the open meth[SEP]"

In [ ]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['ruby']

In [ ]:
encoded_dataset.set_format("torch")

# Define Model

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import CanineForSequenceClassification, AdamW

model = AutoModelForSequenceClassification.from_pretrained("google/canine-s",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 16
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"canine-finetuned-stackoverflow_question_tagger",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_average,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:

encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:

encoded_dataset['train']['input_ids'][10]

tensor([57344,   115,   107,   105,   112,    32,   106,   119,   116,    32,
           97,   117,   116,   104,    32,   100,   117,   114,   105,   110,
          103,    32,   116,   101,   115,   116,   115,    32,    97,   115,
          112,   110,   101,   116,    32,    99,   111,   114,   101,    32,
          119,   101,    98,    32,    97,   112,   105,    32,   105,    32,
           97,    32,   104,    97,   118,   101,    32,    97,    32,   118,
          101,   114,   121,    32,   115,   105,   109,   112,   108,   101,
           32,    97,   112,   112,    32,   119,   105,   116,   104,    32,
          111,   110,   101,    32,   106,   119,   116,    32,    97,   117,
          116,   104,   101,   110,   116,   105,    99,    97,   116,   101,
          100,    32,    99,   111,   110,   116,   114,   111,   108,   108,
          101,   114,    32,   119,   105,   116,   104,    32,   116,   104,
          101,    32,    97,   117,   116,   104,   101,   110, 

In [ ]:

#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7270, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.1971, -0.4634,  0.1905,  0.0193, -0.0960,  0.2269, -0.3823, -0.0027,
         -0.0636,  0.0522,  0.3331, -0.0932,  0.0262,  0.3209, -0.1156,  0.0700,
          0.0070, -0.0650,  0.2788,  0.1959,  0.0326, -0.4568, -0.2518,  0.0461,
         -0.2746,  0.1538, -0.2497,  0.5888,  0.0898,  0.8926, -0.0294, -0.7329,
         -0.2506,  0.1216, -0.1359, -0.0593,  0.2314,  0.5675,  0.1275,  0.4460,
         -0.1219,  0.2482,  0.2276,  0.1456, -0.1175, -0.4055,  0.5479, -0.9486,
          0.2706,  0.2208]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience= 5 , early_stopping_threshold = 0)]
)

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 35,040
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21,900
  Number of trainable parameters = 132,121,394


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.109200,0.096995,0.316997,0.196804
2,0.078600,0.075512,0.464224,0.271918
3,0.066500,0.067618,0.567790,0.310731
4,0.059300,0.063008,0.613154,0.340639
5,0.052400,0.061332,0.639522,0.360959
6,0.047400,0.059763,0.657507,0.368950
7,0.043300,0.059195,0.671227,0.383790


***** Running Evaluation *****
  Num examples = 4380
  Batch size = 16
Saving model checkpoint to canine-finetuned-stackoverflow_question_tagger/checkpoint-2190
Configuration saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-2190/config.json
Model weights saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-2190/pytorch_model.bin
tokenizer config file saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-2190/tokenizer_config.json
Special tokens file saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-2190/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4380
  Batch size = 16
Saving model checkpoint to canine-finetuned-stackoverflow_question_tagger/checkpoint-4380
Configuration saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-4380/config.json
Model weights saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-4380/pytorch_model.bin
tokenizer config file saved in canine-fine

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.109200,0.096995,0.316997,0.196804
2,0.078600,0.075512,0.464224,0.271918
3,0.066500,0.067618,0.567790,0.310731
4,0.059300,0.063008,0.613154,0.340639
5,0.052400,0.061332,0.639522,0.360959
6,0.047400,0.059763,0.657507,0.368950
7,0.043300,0.059195,0.671227,0.383790
8,0.039900,0.059406,0.676050,0.383790
9,0.037200,0.059382,0.675956,0.384018
10,0.035400,0.059195,0.677206,0.387443


***** Running Evaluation *****
  Num examples = 4380
  Batch size = 16
Saving model checkpoint to canine-finetuned-stackoverflow_question_tagger/checkpoint-17520
Configuration saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-17520/config.json
Model weights saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-17520/pytorch_model.bin
tokenizer config file saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-17520/tokenizer_config.json
Special tokens file saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-17520/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4380
  Batch size = 16
Saving model checkpoint to canine-finetuned-stackoverflow_question_tagger/checkpoint-19710
Configuration saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-19710/config.json
Model weights saved in canine-finetuned-stackoverflow_question_tagger/checkpoint-19710/pytorch_model.bin
tokenizer config file saved in can

TrainOutput(global_step=21900, training_loss=0.06128672281901042, metrics={'train_runtime': 13182.1879, 'train_samples_per_second': 26.581, 'train_steps_per_second': 1.661, 'total_flos': 1.151168287113216e+17, 'train_loss': 0.06128672281901042, 'epoch': 10.0})

In [ ]:

trainer.evaluate() #on validation

***** Running Evaluation *****
  Num examples = 4380
  Batch size = 16


{'eval_loss': 0.0591946542263031,
 'eval_f1': 0.6772055888845996,
 'eval_accuracy': 0.38744292237442923,
 'eval_runtime': 64.0582,
 'eval_samples_per_second': 68.375,
 'eval_steps_per_second': 4.277}

In [ ]:

trainer.evaluate(eval_dataset=encoded_dataset['test'])

***** Running Evaluation *****
  Num examples = 4381
  Batch size = 16


{'eval_loss': 0.05945124477148056,
 'eval_f1': 0.6747147534241212,
 'eval_accuracy': 0.3855284181693677,
 'eval_runtime': 63.8668,
 'eval_samples_per_second': 68.596,
 'eval_steps_per_second': 4.29}

In [ ]:
trainer.save_model("/content/drive/MyDrive/Openclassrooms/projet7/CANINE2_multilabel_classification_stackoverflow_question_tagger")

Saving model checkpoint to /content/drive/MyDrive/Openclassrooms/projet7/CANINE2_multilabel_classification_stackoverflow_question_tagger
Configuration saved in /content/drive/MyDrive/Openclassrooms/projet7/CANINE2_multilabel_classification_stackoverflow_question_tagger/config.json
Model weights saved in /content/drive/MyDrive/Openclassrooms/projet7/CANINE2_multilabel_classification_stackoverflow_question_tagger/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Openclassrooms/projet7/CANINE2_multilabel_classification_stackoverflow_question_tagger/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Openclassrooms/projet7/CANINE2_multilabel_classification_stackoverflow_question_tagger/special_tokens_map.json


In [ ]:
#from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Openclassrooms/projet7/my_model_multilabel_classification_oBert_stackoverflow_question_tagger")

loading configuration file /content/drive/MyDrive/Openclassrooms/projet7/my_model_multilabel_classification_oBert_stackoverflow_question_tagger/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/Openclassrooms/projet7/my_model_multilabel_classification_oBert_stackoverflow_question_tagger",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": ".net",
    "1": "ajax",
    "2": "algorithm",
    "3": "android",
    "4": "angular",
    "5": "arrays",
    "6": "asp.net",
    "7": "asp.net-mvc",
    "8": "c",
    "9": "c#",
    "10": "c++",
    "11": "c++11",
    "12": "css",
    "13": "database",
    "14": "django",
    "15": "gcc",
    "16": "html",
    "17": "image",
    "18": "ios",
    "19": "iphone",
    "20": "java",
    "21": "javascript",
    "

In [ ]:
model.eval()

CanineForSequenceClassification(
  (canine): CanineModel(
    (char_embeddings): CanineEmbeddings(
      (HashBucketCodepointEmbedder_0): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_1): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_2): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_3): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_4): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_5): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_6): Embedding(16384, 96)
      (HashBucketCodepointEmbedder_7): Embedding(16384, 96)
      (char_position_embeddings): Embedding(16384, 768)
      (token_type_embeddings): Embedding(16, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (initial_char_encoder): CanineEncoder(
      (layer): ModuleList(
        (0): CanineLayer(
          (attention): CanineAttention(
            (self): CanineSelfAttention(
            

In [ ]:
print('le nombre de paramètre de ce modèle est : ',sum(p.numel() for p in model.parameters()))

le nombre de paramètre de ce modèle est :  132121394


In [ ]:
model.num_parameters()

132121394

# Inference

Let's test the model on a question:

In [ ]:
example = encoded_dataset['test'][17]

In [ ]:
tokenizer.decode(example['input_ids'])

"[CLS]how can i make the win api window more modern looking i ordered programming windows fifth edition a few days ago and started working with it i'm starting to learn the win api however i got a question the windows do not look modern winxpwin vistawin style at all how do i fix this it currently looks like this crap font and all thanks in advance machiel [SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['c', 'c++', 'windows']

In [ ]:
text = "how can i make the win api window more modern looking i ordered programming windows fifth edition a few days ago and started working with it i'm starting to learn the win api however i got a question the windows do not look modern winxpwin vistawin style at all how do i fix this it currently looks like this crap font and all thanks in advance machiel"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.



To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
import time
start_time= time.time() # set the time at which inference started
logits = outputs.logits
logits.shape
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
stop_time=time.time()
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
duration =stop_time - start_time
hours = duration // 3600
minutes = (duration - (hours * 3600)) // 60
seconds = duration - ((hours * 3600) + (minutes * 60))
print(f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes,{seconds:4.2f} seconds)', flush=True) # print out inferenceduration time

['c++', 'windows']
training elapsed time was 0.0 hours,  0.0 minutes,0.01 seconds)


In [ ]:
example_2 = encoded_dataset['test'][7]

In [ ]:
tokenizer.decode(example_2['input_ids'])

"[CLS]hide virtual function with nonvirtual override having the output is but i expected i didn't declare with the modifier so i expect the function call to be determined by the static type no polymorphism why is being called is it possible to provide a nonvirtual function in a derived class that hides the virtual function in the base what signature should the derived member function have so that calls it and not [SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]"

In [ ]:
[id2label[idx] for idx, label in enumerate(example_2['labels']) if label == 1.0]

['c++']

In [ ]:
text = "hide virtual function with nonvirtual override having the output is but i expected i didn't declare with the modifier so i expect the function call to be determined by the static type no polymorphism why is being called is it possible to provide a nonvirtual function in a derived class that hides the virtual function in the base what signature should the derived member function have so that calls it and not "

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
import time
start_time= time.time() # set the time at which inference started
logits = outputs.logits
logits.shape
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
stop_time=time.time()
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
duration =stop_time - start_time
hours = duration // 3600
minutes = (duration - (hours * 3600)) // 60
seconds = duration - ((hours * 3600) + (minutes * 60))
print(f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes,{seconds:4.2f} seconds)', flush=True) # print out inferenceduration time

['c++']
training elapsed time was 0.0 hours,  0.0 minutes,0.01 seconds)


In [ ]:
example_3 = encoded_dataset['test'][10]

In [ ]:
tokenizer.decode(example_3['input_ids'])

'[CLS]url encoded forward slashes breaking my codeigniter app im trying to create a url string that works like this so in other words i then retrieve the url with the forward slashes in the url will of course be a problem accessing uri segments so ill go ahead and url encode the url portion but now i just get a saying it appears that my url encoding of forward slashes breaks cis uri parser what can i do to get around this problem [SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [ ]:
[id2label[idx] for idx, label in enumerate(example_3['labels']) if label == 1.0]

['php']

In [ ]:
text = "url encoded forward slashes breaking my codeigniter app im trying to create a url string that works like this so in other words i then retrieve the url with the forward slashes in the url will of course be a problem accessing uri segments so ill go ahead and url encode the url portion but now i just get a saying it appears that my url encoding of forward slashes breaks cis uri parser what can i do to get around this problem"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
import time
start_time= time.time() # set the time at which inference started
logits = outputs.logits
logits.shape
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
stop_time=time.time()
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
duration =stop_time - start_time
hours = duration // 3600
minutes = (duration - (hours * 3600)) // 60
seconds = duration - ((hours * 3600) + (minutes * 60))
print(f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes,{seconds:4.2f} seconds)', flush=True) # print out inferenceduration time

['python']
training elapsed time was 0.0 hours,  0.0 minutes,0.01 seconds)
